In [ ]:
import pandas as pd, numpy as np
print("pandas: ",pd.__version__)
print("numpy: ",np.__version__)

pandas:  2.2.2
numpy:  2.0.2


In [ ]:
from google.colab import files
uploaded = files.upload()
csv_path = list(uploaded.keys())[0]
print ("Archivo cargado: ", csv_path)

Saving Sample - Superstore.csv to Sample - Superstore.csv
Archivo cargado:  Sample - Superstore.csv


In [ ]:
read_kwargs = dict(encoding = "latin-1", sep = ",", decimal=".", thousands=None)
raw = pd.read_csv(csv_path, **read_kwargs)
print("Raw Shape: ",raw.shape)
raw.head(3)

Raw Shape:  (9994, 21)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


In [ ]:
def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    col_map = {
        # EN → estándar
        'Order Date':'OrderDate', 'Ship Date':'ShipDate', 'Ship Mode':'ShipMode',
        'Customer ID':'CustomerID', 'Customer Name':'CustomerName', 'Segment':'Segment',
        'Country':'Country', 'City':'City', 'State':'StateProvince', 'Postal Code':'PostalCode', 'Region':'Region',
        'Product ID':'ProductID', 'Category':'Category', 'Sub-Category':'SubCategory',
        'Product Name':'ProductName', 'Sales':'Sales', 'Quantity':'Quantity',
        'Discount':'Discount', 'Profit':'Profit', 'Order ID':'OrderID',
        # ES → estándar
        'Fecha Pedido':'OrderDate', 'Fecha de Pedido':'OrderDate',
        'Fecha Envío':'ShipDate', 'Modo de envío':'ShipMode',
        'ID Cliente':'CustomerID', 'Nombre Cliente':'CustomerName', 'Segmento':'Segment',
        'País':'Country', 'Ciudad':'City', 'Estado/Provincia':'StateProvince',
        'Código Postal':'PostalCode', 'Región':'Region',
        'ID Producto':'ProductID', 'Categoría':'Category', 'Subcategoría':'SubCategory',
        'Nombre del producto':'ProductName',
        'Ventas':'Sales', 'Cantidad':'Quantity', 'Descuento':'Discount',
        'Beneficio':'Profit', 'ID Pedido':'OrderID'
    }
    ren = {c: col_map.get(c, c) for c in df.columns}
    df = df.rename(columns=ren)
    return df

df = standardize_columns(raw)
sorted(df.columns)


['Category',
 'City',
 'Country',
 'CustomerID',
 'CustomerName',
 'Discount',
 'OrderDate',
 'OrderID',
 'PostalCode',
 'ProductID',
 'ProductName',
 'Profit',
 'Quantity',
 'Region',
 'Row ID',
 'Sales',
 'Segment',
 'ShipDate',
 'ShipMode',
 'StateProvince',
 'SubCategory']

In [ ]:
from pandas.api.types import is_numeric_dtype


for c in ['OrderDate','ShipDate']:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce', dayfirst=False)


for c in ['Sales','Quantity','Discount','Profit']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')


for c in df.select_dtypes(include='object').columns:
    df[c] = df[c].astype(str).str.strip()


essential = [c for c in ['OrderID','OrderDate','CustomerID','ProductID','Sales','Quantity'] if c in df.columns]
df = df.dropna(subset=essential).copy()


for c in ['CustomerName','Segment','City','StateProvince','Country','Region','PostalCode','Category','SubCategory','ProductName','ShipMode']:
    if c in df.columns:
        df[c] = df[c].fillna('Unknown')

for c in ['Discount','Profit']:
    if c in df.columns:
        df[c] = df[c].fillna(0)


if {'OrderID','ProductID'}.issubset(df.columns):
    before = len(df)
    df = df.drop_duplicates(subset=['OrderID','ProductID'], keep='first')
    print("Duplicados removidos:", before - len(df))

df.shape


Duplicados removidos: 8


(9986, 21)

In [ ]:


if 'Discount' in df.columns:
    df['Discount'] = np.where(df['Discount']>1, df['Discount']/100.0, df['Discount']).clip(0, 0.9)


if {'Sales','Quantity'}.issubset(df.columns):
    df['UnitPrice'] = np.where(df['Quantity']>0, df['Sales']/df['Quantity'], np.nan)

if {'Profit','Sales'}.issubset(df.columns):
    df['MarginPct'] = np.where(df['Sales']!=0, df['Profit']/df['Sales'], np.nan)


if 'OrderDate' in df.columns:
    df['Year'] = df['OrderDate'].dt.year
    df['MonthNum'] = df['OrderDate'].dt.month
    df['YearMonth'] = df['OrderDate'].dt.to_period('M').astype(str)

df[['Sales','Quantity','Discount','UnitPrice','MarginPct']].describe().T


,count,mean,std,min,25%,50%,75%,max
Sales,9986.0,229.872779,623.477775,0.444,17.248,54.384,209.9175,22638.48
Quantity,9986.0,3.789305,2.225286,1.000,2.000,3.000,5.0000,14.00
Discount,9986.0,0.156258,0.206499,0.000,0.000,0.200,0.2000,0.80
UnitPrice,9986.0,60.914405,142.974063,0.336,5.448,16.267,63.9255,3773.08
MarginPct,9986.0,0.120185,0.466894,-2.750,0.075,0.270,0.3625,0.50


In [ ]:
def iqr_cap(s: pd.Series, k=1.5):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - k*iqr, q3 + k*iqr
    return s.clip(lower, upper)

for c in ['Sales','Quantity','Profit','UnitPrice']:
    if c in df.columns and is_numeric_dtype(df[c]):
        df[c] = iqr_cap(df[c], k=1.5)

df[['Sales','Quantity','Profit','UnitPrice']].describe().T


,count,mean,std,min,25%,50%,75%,max
Sales,9986.0,140.227710,168.806295,0.4440,17.248,54.38400,209.9175,498.92175
Quantity,9986.0,3.753054,2.102638,1.0000,2.000,3.00000,5.0000,9.50000
Profit,9986.0,16.050154,29.475691,-39.7107,1.728,8.64135,29.3538,70.79250
UnitPrice,9986.0,42.459596,50.533792,0.3360,5.448,16.26700,63.9255,151.64175


In [ ]:
print("Nulos por columna (top 15):\n", df.isna().sum().sort_values(ascending=False).head(15))

if {'YearMonth','Sales'}.issubset(df.columns):
    mensual = df.groupby('YearMonth', as_index=False)['Sales'].sum().sort_values('YearMonth')
    display(mensual.tail())

if {'ProductName','Sales'}.issubset(df.columns):
    top_prod = df.groupby('ProductName', as_index=False)['Sales'].sum().sort_values('Sales', ascending=False).head(10)
    display(top_prod)


Nulos por columna (top 15):
 Row ID           0
OrderID          0
OrderDate        0
ShipDate         0
ShipMode         0
CustomerID       0
CustomerName     0
Segment          0
Country          0
City             0
StateProvince    0
PostalCode       0
Region           0
ProductID        0
Category         0
dtype: int64


,YearMonth,Sales
43,2017-08,33589.71800
44,2017-09,58617.35550
45,2017-10,39325.52145
46,2017-11,66228.33925
47,2017-12,59881.51655


,ProductName,Sales
1363,SAFCO Arco Folding Chair,6038.77750
1551,Tennsco Double-Tier Lockers,5619.27350
368,Bretford Rectangular Conference Table Tops,5591.02550
839,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",5539.40975
686,GBC DocuBind TL300 Electric Binding System,5488.13925
872,"Hot File 7-Pocket, Floor Stand",5418.33975
1549,Tennsco 6- and 18-Compartment Lockers,5179.73775
428,Chromcraft Round Conference Tables,5172.15550
650,Fellowes PB500 Electric Punch Plastic Comb Bin...,4989.21750
842,Hon Every-Day Series Multi-Task Chairs,4893.67800


In [ ]:


clean_path_std   = "/content/superstore_clean.csv"
clean_path_excel = "/content/superstore_clean_excel.csv"
dict_path        = "/content/data_dictionary.csv"
df.to_csv(clean_path_std, index=False)
df.to_csv(clean_path_excel, index=False, sep=';', decimal=',')

pd.DataFrame({
    "column": df.columns,
    "dtype": [str(df[c].dtype) for c in df.columns],
    "nulls": [int(df[c].isna().sum()) for c in df.columns]
}).to_csv(dict_path, index=False)

print("Listo:")
print(" - CSV estándar:", clean_path_std)
print(" - CSV Excel-friendly:", clean_path_excel)
print(" - Data dictionary:", dict_path)

files.download(clean_path_std)
files.download(clean_path_excel)
files.download(dict_path)

Listo:
 - CSV estándar: /content/superstore_clean.csv
 - CSV Excel-friendly: /content/superstore_clean_excel.csv
 - Data dictionary: /content/data_dictionary.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>